<img width="8%" alt="Google Sheets.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Sheets.png" style="border-radius: 15%">

# Google Sheets - Update deals database

**Tags:** #googlesheets #gsheet #data #naas_drivers #sales-engine #automation #deals

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook updates "Deals" database.

## Input

### Import libraries

In [ ]:
from naas_drivers import gsheet
import pandas as pd
import os
from datetime import date
import naas_data_product
import time

### Setup variables
**Inputs**
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name_input`: Google Sheets sheet name storing growth data.

**Outputs**
- `output_dir`: Output directory to save file to.
- `file_sales`: Output file name to save as picke.
- `sheet_name_output`: Google Sheets sheet name storing sales qualified leads.

In [ ]:
# Inputs
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
sheet_name_input = "CONTACTS"

# Outputs
output_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "sales-engine", date.today().isoformat())
file_deals = "deals"
sheet_name_output = "DEALS"

## Model

### Get deals

In [ ]:
df_init = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name_output)
if not isinstance(df_init, pd.DataFrame):
    df_init = pd.DataFrame()
print("- Deals (init):", len(df_init))
# df_init.head(3)

### Get contacts view

In [ ]:
df_contacts = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name_input)
if not isinstance(df_contacts, pd.DataFrame):
    df_contacts = pd.DataFrame()
print("- Contacts view:", len(df_contacts))
# df_contacts.head(3)

### Get data from ENTITY spreadsheet

In [ ]:
df_gsheet = gsheet.connect(spreadsheet_url).get(sheet_name="ENTITY").fillna("NA")
# df_gsheet.head(1)

### Create deals database

In [ ]:
def get_list(df, column):
    filters = []
    values = df.loc[0, column].split(",")
    if len(values) > 0:
        filters = [v.strip() for v in values]
    return filters

def create_db_deals(
    df_init,
    df_gsheet
):
    # Get filters from gsheet
    filter_seniority = get_list(df_gsheet, "ICP_SENIORITY")
    filter_department = get_list(df_gsheet, "ICP_DEPARTMENT")
    filter_org_staff_range = get_list(df_gsheet, "ICP_ORG_STAFF_RANGE")
    filter_org_industry = get_list(df_gsheet, "ICP_ORG_INDUSTRY")
    print("Seniority:", filter_seniority)
    print()
    print("Department:", filter_department)
    print()
    print("Staff Range:", filter_org_staff_range)
    print()
    print("Industry:", filter_org_industry)
    print()
    
    # Init
    df = df_init.copy()
    
    # Filter data
    if filter_org_industry[0] == "NA":
        filter_org_industry = df["ORG_INDUSTRY"].unique().tolist()
    df = df[
        (df["LEAD_STATUS"].isin(["Sales Qualified Lead"])) &
        (df["PEOPLE_SENIORITY"].isin(filter_seniority)) &
        (df["PEOPLE_DEPARTMENT"].isin(filter_department)) & 
        (df["ORG_STAFF_RANGE_NAME"].isin(filter_org_staff_range)) & 
        (df["ORG_INDUSTRY"].isin(filter_org_industry))
    ]
    df["SCENARIO"] = pd.to_datetime(df["SQL_DATE"].str[:19]).dt.strftime("W%W-%Y")
    df["DEAL_NAME"] = df["PEOPLE_FULLNAME"] + " (" + df["ORG_NAME"] + ")"
    df["DEAL_SCORE"] = df["PEOPLE_INTERACTION_SCORE"].astype(int) + df["ORG_INTERACTION_SCORE"].astype(int)
    df["CRM_DEAL_ID"] = "TBD"
    
    # Cleaning
    to_order = [
        "SCENARIO",
        "DEAL_NAME",
        "SQL_DATE",
        "DEAL_SCORE",
        "PEOPLE_FULLNAME",
        "PEOPLE_OCCUPATION",
        "PEOPLE_SENIORITY",
        "PEOPLE_DEPARTMENT",
        "MESSAGING_OPTIONS",
        "ORG_NAME",
        "ORG_STAFF_RANGE_NAME",
        "ORG_INDUSTRY",
        'ORG_COUNTRY',
        'PEOPLE_PROFILE_URL',
        'ORG_LINKEDIN_URL',
        "CRM_CONTACT_ID",
        "CRM_ORG_ID",
        "CRM_DEAL_ID"
    ]
    to_rename = {
        "SQL_DATE": "DEAL_CREATED_DATE",
    }
    df = df[to_order].rename(columns=to_rename).sort_values(by="DEAL_CREATED_DATE", ascending=False)
    return df.reset_index(drop=True)

db_deals = create_db_deals(
    df_contacts,
    df_gsheet
)
print("- Deals:", len(db_deals))
db_deals.head(3)

## Output

### Save data

In [ ]:
pdump(output_dir, db_deals, file_deals)

### Send data to Google Sheets spreadsheet

In [ ]:
df_check = pd.concat([db_deals.astype(str), df_init.astype(str)]).drop_duplicates(keep=False)
if len(df_check) > 0:
    gsheet.connect(spreadsheet_url).send(sheet_name=sheet_name_output, data=db_deals, append=False)
else:
    print("Noting to update in Google Sheets!")   